# Imports

In [ ]:
import numpy as np
import sys

from aie.dialects.aie import *
from aie.dialects.aiex import *
from aie.extras.context import mlir_mod_ctx
from aie.helpers.dialects.func import func
from aie.iron.controlflow import range_

from aie.utils import NPUCallable, DefaultNPURuntime, get_current_device
import aie.iron as iron
from aie.iron.device import NPU2
import aie.utils.test as test_utils

# Design Configurations

In [ ]:
VECTOR_SIZE = 4096

# Passthrough PyKernel Design

In [ ]:
def passthroughKernel(vector_size):
    if vector_size % 64 != 0 or vector_size < 512:
        raise ValueError(
            "Vector size must be a multiple of 64 and greater than or equal to 512"
        )
    N = vector_size
    lineWidthInBytes = N // 4  # chop input in 4 sub-tensors

    d = get_current_device()
    if d is None:
        raise RuntimeError("Could not determine current device")

    @device(d.resolve())
    def device_body():
        # define types
        line_ty = np.ndarray[(lineWidthInBytes,), np.dtype[np.uint8]]

        # AIE Core Python Function declarations
        @func(emit=True)
        def passThroughLine(input: line_ty, output: line_ty, lineWidth: np.int32):
            for i in range_(lineWidth):
                output[i] = input[i]

        # Tile declarations
        ShimTile = tile(0, 0)
        ComputeTile2 = tile(0, 2)

        # AIE-array data movement with object fifos
        of_in = object_fifo("in", ShimTile, ComputeTile2, 2, line_ty)
        of_out = object_fifo("out", ComputeTile2, ShimTile, 2, line_ty)

        # Set up compute tiles

        # Compute tile 2
        @core(ComputeTile2)
        def core_body():
            for _ in range_(sys.maxsize):
                elemOut = of_out.acquire(ObjectFifoPort.Produce, 1)
                elemIn = of_in.acquire(ObjectFifoPort.Consume, 1)
                passThroughLine(elemIn, elemOut, lineWidthInBytes)
                of_in.release(ObjectFifoPort.Consume, 1)
                of_out.release(ObjectFifoPort.Produce, 1)

        #    print(ctx.module.operation.verify())

        vector_ty = np.ndarray[(N,), np.dtype[np.uint8]]

        @runtime_sequence(vector_ty, vector_ty, vector_ty)
        def sequence(inTensor, outTensor, notUsed):
            in_task = shim_dma_single_bd_task(
                of_in, inTensor, sizes=[1, 1, 1, N], issue_token=True
            )
            out_task = shim_dma_single_bd_task(
                of_out, outTensor, sizes=[1, 1, 1, N], issue_token=True
            )

            dma_start_task(in_task, out_task)
            dma_await_task(in_task, out_task)

# Generate MLIR, print, and save to file

In [ ]:
with mlir_mod_ctx() as ctx:
    passthroughKernel(VECTOR_SIZE)
    # Warning: Do not change file name without changing compilation commands below
    with open("notebook_aie.mlir", "w") as f:
        print(ctx.module)
        print(ctx.module, file=f)

# Compile the design using `aiecc`

In [ ]:
!mkdir notebook_build
!cd notebook_build && aiecc.py --aie-generate-xclbin --aie-generate-npu-insts --no-compile-host --no-xchesscc --no-xbridge --xclbin-name=notebook.xclbin --npu-insts-name=notebook_insts.bin ../notebook_aie.mlir

# Test the Design use pyxrt

In [ ]:
dtype = np.uint8

npu_kernel = NPUCallable(
    "notebook_build/notebook.xclbin",
    "notebook_build/notebook_insts.bin",
    kernel_name="MLIR_AIE",
)
kernel_handle = DefaultNPURuntime.load(npu_kernel)

input = np.arange(1, VECTOR_SIZE + 1, dtype=dtype)
in_tensor = iron.tensor(input, dtype=dtype)
out_tensor = iron.zeros(VECTOR_SIZE, dtype=dtype)

buffers = [in_tensor, out_tensor]

print("Running...")
DefaultNPURuntime.run(kernel_handle, buffers)
aie_output = out_tensor.numpy()

# Copy output results and verify they are correct
errors = 0
print("Verifying results ...")
e = np.equal(input, aie_output)
errors = np.size(e) - np.count_nonzero(e)

if not errors:
    print("\nPASSED!\n")
else:
    print("\nError count: ", errors)
    print("\nFailed.\n")